# Project: Building RAG Chatbots for Technical Documentation

A well-known car manufacturer that is looking at implementing large language models (LLMs) into vehicles to help drivers. Experiment with integrating car manuals with an LLM to create a chatbot that understands the situation. The hope is that this chatbot can be connected to a text-to-speech program to read its answers out loud.

As a test, I'll use several pages from a car manual that explain car warning messages, their meanings, and what to do about them. This specific manual, saved as an HTML file named `mg-zs-warning-messages.html`, is for an MG ZS, which is a small SUV. Using what I've learned about `LLMs` and `LangChain`, I will build the chatbot using a technique called `Retrieval Augmented Generation (RAG)`.

> **Retrieval Augmented Generation (RAG):** is a technique that gives large language models (LLMs) the ability to access and use up-to-date, external information to create more accurate and relevant responses.

> **LangChain:** is a set of tools that helps developers create powerful applications by connecting large language models (LLMs) with other sources of data.





## Necessary packages

In [ ]:
# Install the necessary packages
import subprocess
import pkg_resources

def install_if_needed(package, version):
    '''Function to ensure that the libraries used are consistent to avoid errors.'''
    try:
        pkg = pkg_resources.get_distribution(package)
        if pkg.version != version:
            raise pkg_resources.VersionConflict(pkg, version)
    except (pkg_resources.DistributionNotFound, pkg_resources.VersionConflict):
        subprocess.check_call(["pip", "install", f"{package}=={version}"])

install_if_needed("langchain-core", "0.3.72")
install_if_needed("langchain-openai", "0.3.28")
install_if_needed("langchain-community", "0.3.27")
install_if_needed("unstructured", "0.18.11")
install_if_needed("langchain-chroma", "0.2.5")
install_if_needed("langchain-text-splitters", "0.3.9")

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [ ]:
!python3 -m pip install --upgrade pip

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
# Import the required packages
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.document_loaders import UnstructuredHTMLLoader
from langchain_core.runnables import RunnablePassthrough
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
import os

## Loading Data

Load the HTML as a LangChain document loader

In [ ]:
loader = UnstructuredHTMLLoader(file_path="data/mg-zs-warning-messages.html")
car_docs = loader.load()

In [ ]:
# Display the contents of the car_docs documents
for i, doc in enumerate(car_docs):
    print(f"Document {i+1}:\n{doc}\n")

Document 1:
page_content='Warning Message Procedure Cruise Control Fault Indicates that the cruise control system has detected a fault. Please consult an MG Authorised Repairer as soon as possible. Active Speed Limiter Fault Indicates that the active speed limit system has detected a fault. Contact an MG Authorised Repairer as soon as possible. Engine Coolant Temperature High High engine coolant temperature could result in severe damage. As soon as conditions permit, safely stop the vehicle and switch off the engine and contact an MG Authorised Repairer immediately. Engine Coolant Temperature Sensor Fault Indicates that the engine coolant temperature sensor has failed. As soon as conditions permit, safely stop the vehicle and switch off the engine and contact an MG Authorised Repairer immediately.

Warning Message Procedure Low Oil Pressure Indicates that the oil pressure is too low, which may result in severe engine damage. As soon as safety permits, stop the car, switch off the engin

## Load the models

In [ ]:
# Load the models required to complete the exercise
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
embeddings = OpenAIEmbeddings(model="text-embedding-3-small", openai_api_key=os.environ["OPENAI_API_KEY"])

## Splitting Data

The loaded document is split into smaller, manageable chunks  to facilitate efficient retrieval.

In [ ]:
# Initialize the RecursiveCharacterTextSplitter with desired chunk size and overlap
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

# Split the loaded documents into chunks
car_docs_chunks = text_splitter.split_documents(car_docs)

In [ ]:
# Display the contents of the car_docs_chunks
for i, chunk in enumerate(car_docs_chunks):
    print(f"Chunk {i+1}:\n{chunk}\n")

Chunk 1:
page_content='Warning Message Procedure Cruise Control Fault Indicates that the cruise control system has detected a fault. Please consult an MG Authorised Repairer as soon as possible. Active Speed Limiter Fault Indicates that the active speed limit system has detected a fault. Contact an MG Authorised Repairer as soon as possible. Engine Coolant Temperature High High engine coolant temperature could result in severe damage. As soon as conditions permit, safely stop the vehicle and switch off the engine and contact an MG Authorised Repairer immediately. Engine Coolant Temperature Sensor Fault Indicates that the engine coolant temperature sensor has failed. As soon as conditions permit, safely stop the vehicle and switch off the engine and contact an MG Authorised Repairer immediately.' metadata={'source': 'data/mg-zs-warning-messages.html'}

Chunk 2:
page_content='Warning Message Procedure Low Oil Pressure Indicates that the oil pressure is too low, which may result in severe

## Setting up Vector Store
The document chunks and their embeddings are stored in a Chroma vector database for efficient similarity search.

In [ ]:
chroma_store = Chroma.from_documents(
    documents=car_docs_chunks,
    embedding=embeddings
)

Retriever to retrieve relevant documents from the vector store

In [ ]:
retriever = chroma_store.as_retriever()

## Initializing LLM and Prompt

A ChatOpenAI model (gpt-4o-mini) and a ChatPromptTemplate are initialized to generate responses.

In [ ]:
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0  # Default temperature set to 0 for less creative outputs
)

In [ ]:
# Create an instance of the ChatPromptTemplate class and use the .from_template() method
prompt_template = ChatPromptTemplate.from_template(
    "Given the following context: {context}, please provide a detailed response."
)

## Defining RAG Chain

A RAG chain is defined using LangChain Expression Language (LCEL), connecting the retriever (vector store), the prompt template, and the LLM.

In [ ]:
# Define the RAG chain using LangChain Expression Language (LCEL)
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt_template
    | llm
)

## Invoking RAG Chain
The RAG chain is invoked with a user query to retrieve relevant information from the manual and generate a context-aware response.

In [ ]:
# Define the user query
user_query = "The Gasoline Particular Filter Full warning has appeared. What does this mean and what should I do about it?"

# Invoke chain with the user query
answer = rag_chain.invoke(user_query).content

# Display
print(answer)

The provided context contains a series of warning messages related to the MG ZS vehicle, detailing various system faults and issues that may arise during operation. Each warning message is accompanied by a recommended procedure for addressing the issue, typically advising the driver to consult an MG Authorised Repairer. Below is a summary of the key warning messages and their implications:

1. **Stop Start System Fault**:
   - **Description**: This warning indicates that the Stop/Start intelligent fuel-saving system has detected a fault.
   - **Action**: The driver is advised to consult an MG Authorised Repairer as soon as possible.

2. **Clutch Switch Fault**:
   - **Description**: This message indicates a fault detected in the clutch switch.
   - **Action**: Similar to the previous warning, the driver should consult an MG Authorised Repairer promptly.

3. **Gasoline Particular Filter Full**:
   - **Description**: This warning signifies that the gasoline particulate filter is full.
  